In [1]:
import numpy as np
import pandas as pd


class ClinicalTrial:
    rho: float  # Relative importance between first and second moments
    w: np.ndarray  # Normalized patient covariates

    def __init__(self, w: np.ndarray, rho=0.5) -> None:
        self.rho = rho
        # Normalize covariates
        self.w = (w - np.mean(w, axis=0)) / np.std(w, axis=0)

    def assert_valid(self, group1: np.ndarray, group2: np.ndarray) -> None:
        """
        Checks if the patient constraints are met.

        Arguments (where n is the number of patients):
            - group1: np.ndarray(size = n) => Binary array of patients belonging to group 1
            - group2: np.ndarray(size = n) => Binary array of patients belonging to group 2
        Throws an AssertionError if the constraints are not met.
        """
        group_size = int(self.w.shape[0] / 2)
        # constraint 1: number of people in each group
        assert (
            np.sum(group1) == group_size
        ), f"Each group should have {group_size} patients"
        # contraint 2: every patient is in one group
        assert (
            group1 + group2 == 1
        ).all(), "Every patient needs to be assigned to one group"

    def discrepancy(self, group1: np.ndarray, group2: np.ndarray) -> float:
        """
        Calculates discrepancy between patient groups.

        Arguments (where n is the number of patients):
            - group1: np.ndarray(size = n) => Binary array of patients belonging to group 1
            - group2: np.ndarray(size = n) => Binary array of patients belonging to group 2
        Returns:
            - float => Value of discrepancy measure for group1 and group2
        """
        # Check that all the constraints are being met
        self.assert_valid(group1, group2)

        # Order of the groups is arbitrary
        if group1[0] == 0:
            group1, group2 = group2, group1

        n, r = self.w.shape

        # Calculate mean values for each covariate
        Mu = []
        for i in range(r):
            Mu.append(self.w[:, i].dot(group1 - group2) / n)

        # Calculate second moments (variance and covariance)
        Var_ii = []  # variance
        Var_ij = []  # covariance

        for i in range(r):
            for j in range(i, r):
                if i == j:
                    Var_ii.append((self.w[:, i] ** 2).dot(group1 - group2) / n)
                else:
                    Var_ij.append(
                        (self.w[:, i] * self.w[:, j]).dot(group1 - group2) / n
                    )
        # print(            np.sum(np.abs(Mu)))
        # Calculate final discrepancy
        discrepancy = (
            np.sum(np.abs(Mu))
            + self.rho * np.sum(np.abs(Var_ii))
            + 2 * self.rho * np.sum(np.abs(Var_ij))
        )
        return discrepancy

    def __repr__(self) -> str:
        return f"ClinicalTrial(w=np.ndarray(shape={self.w.shape}), rho={self.rho})"

data = pd.read_csv('ingenii-clinical-trial/data/pbc.csv')
w = data.sample(100, random_state=np.random.default_rng(42)).values
ws = (w - np.mean(w, axis=0)) / np.std(w, axis=0)
n, r = ws.shape

In [2]:
import dimod
# N1 = round(np.ceil(np.sum(np.argsort(ws[:,0])[::-1][:int(n/2)])))  # number of slack variables
# N2 = round(np.ceil(np.sum(np.argsort(ws[:,1])[::-1][:int(n/2)])))  # number of slack variables
# N3 = round(np.ceil(np.sum(np.argsort(ws[:,2])[::-1][:int(n/2)])))  # number of slack variables
# N12 = round(np.ceil(np.sum(np.argsort(ws[:,0]*ws[:,1])[::-1][:int(n/2)])))  # number of slack variables
# N13 = round(np.ceil(np.sum(np.argsort(ws[:,0]*ws[:,2])[::-1][:int(n/2)])))  # number of slack variables
# N23 = round(np.ceil(np.sum(np.argsort(ws[:,1]*ws[:,2])[::-1][:int(n/2)])))  # number of slack variables
# Us = {'U1':N1, 'U2':N2, 'U3':N3, 'U4':N12, 'U5':N13, 'U6':N23}

In [3]:
cqm = dimod.ConstrainedQuadraticModel()

In [4]:
xs = {}
for i in range(n):
    xs['x'+str(i+1)] = 0
x = [dimod.Binary(f"x_{j+1}") for j in range(n)]

In [5]:
cqm.add_constraint(sum(x[i] for i in range(n)) == int(n/2), label=f'all_patients_allocation')

'all_patients_allocation'

In [6]:
Us = {'U1':100/27, 'U2':100/27, 'U3':100/27, 'U4':100/27, 'U5':100/27, 'U6':100/27, 'U7':100/27, 'U8':100/27, 'U9':100/27}
quantities = [dimod.Real(f'{U}') for U in Us.keys()]

In [7]:
for ind, U in enumerate(Us.keys()):
    # quantities[ind].set_upper_bound(U, 1)
    quantities[ind].set_upper_bound(U, Us[U])

In [8]:
# Us_bin = {}
# for i in range(6):
#     Us_bin['U'+str(i+1)] = round(np.ceil(np.log2(Us['U'+str(i+1)])))
# slacks = []
# for ind, U in enumerate(Us_bin.keys()):
#   slacks.append([dimod.Binary(f"{U}_{j+1}") for j in range(Us_bin[U])])

In [9]:
# cqm.add_constraint(sum([ws[i,0]*x[i] for i in range(n)]) - sum(slacks[0]) <= 0, label="mu1")
# cqm.add_constraint(sum([ws[i,1]*x[i] for i in range(n)]) - sum(slacks[1]) <= 0, label="mu2")
# cqm.add_constraint(sum([ws[i,2]*x[i] for i in range(n)]) - sum(slacks[2]) <= 0, label="mu3")
# cqm.add_constraint(sum([ws[i,0]*ws[i,1]*x[i] for i in range(n)]) - sum(slacks[3]) <= 0, label="rho12")
# cqm.add_constraint(sum([ws[i,0]*ws[i,2]*x[i] for i in range(n)]) - sum(slacks[4]) <= 0, label="rho13")
# cqm.add_constraint(sum([ws[i,1]*ws[i,2]*x[i] for i in range(n)]) - sum(slacks[5]) <= 0, label="rho23")

In [10]:
# square_cost = sum([1/2*(ws[i,0]**2+ws[i,1]**2+ws[i,2]**2)*x[i] for i in range(n)])
# cqm.set_objective(sum(quantities) + square_cost)
cqm.set_objective(sum(quantities))

In [11]:
cqm.add_constraint(sum([ws[i,0]*x[i] for i in range(n)]) - quantities[0] <= 0, label="mu1")
cqm.add_constraint(sum([ws[i,1]*x[i] for i in range(n)]) - quantities[1] <= 0, label="mu2")
cqm.add_constraint(sum([ws[i,2]*x[i] for i in range(n)]) - quantities[2] <= 0, label="mu3")
cqm.add_constraint(sum([ws[i,0]*ws[i,1]*x[i] for i in range(n)]) - quantities[3] <= 0, label="rho12")
cqm.add_constraint(sum([ws[i,0]*ws[i,2]*x[i] for i in range(n)]) - quantities[4] <= 0, label="rho13")
cqm.add_constraint(sum([ws[i,1]*ws[i,2]*x[i] for i in range(n)]) - quantities[5] <= 0, label="rho23")

cqm.add_constraint(sum([ws[i,0]*ws[i,0]*x[i] for i in range(n)]) - quantities[6] <= 0, label="rho11")
cqm.add_constraint(sum([ws[i,1]*ws[i,1]*x[i] for i in range(n)]) - quantities[7] <= 0, label="rho22")
cqm.add_constraint(sum([ws[i,2]*ws[i,2]*x[i] for i in range(n)]) - quantities[8] <= 0, label="rho33")

cqm.add_constraint(- sum([ws[i,0]*x[i] for i in range(n)]) - quantities[0] <= 0, label="mu1-")
cqm.add_constraint(- sum([ws[i,1]*x[i] for i in range(n)]) - quantities[1] <= 0, label="mu2-")
cqm.add_constraint(- sum([ws[i,2]*x[i] for i in range(n)]) - quantities[2] <= 0, label="mu3-")
cqm.add_constraint(- sum([ws[i,0]*ws[i,1]*x[i] for i in range(n)]) - quantities[3] <= 0, label="rho12-")
cqm.add_constraint(- sum([ws[i,0]*ws[i,2]*x[i] for i in range(n)]) - quantities[4] <= 0, label="rho13-")
cqm.add_constraint(- sum([ws[i,1]*ws[i,2]*x[i] for i in range(n)]) - quantities[5] <= 0, label="rho23-")

cqm.add_constraint(- sum([ws[i,0]*ws[i,0]*x[i] for i in range(n)]) - quantities[6] <= 0, label="rho11-")
cqm.add_constraint(- sum([ws[i,1]*ws[i,1]*x[i] for i in range(n)]) - quantities[7] <= 0, label="rho22-")
cqm.add_constraint(- sum([ws[i,2]*ws[i,2]*x[i] for i in range(n)]) - quantities[8] <= 0, label="rho33-")

'rho33-'

In [12]:
list(cqm.constraints.keys())

['all_patients_allocation',
 'mu1',
 'mu2',
 'mu3',
 'rho12',
 'rho13',
 'rho23',
 'rho11',
 'rho22',
 'rho33',
 'mu1-',
 'mu2-',
 'mu3-',
 'rho12-',
 'rho13-',
 'rho23-',
 'rho11-',
 'rho22-',
 'rho33-']

In [13]:
len(cqm.variables)

109

In [14]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token='DEV-100ec3ead35fe44b2c0f1a7c0ca586059b8c4d45')                        
sampleset = sampler.sample_cqm(cqm, time_limit=60*60)                    
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)   
print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))    

SolverFailureError: Problem not accepted because user has insufficient remaining solver access time in project DEV

In [15]:
# from neal import SimulatedAnnealingSampler
# sampler = SimulatedAnnealingSampler()                 
# qubo_sample = sampler.sample_qubo(cqm, num_reads=100)
# print("objective function:", qubo_sample.first.energy, "\n")
# qubo_solution = qubo_sample.first.sample
# print("qubo solution:", qubo_solution, "\n")               
# feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)   
# print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))    

In [16]:
best = feasible_sampleset.first

ValueError: SampleSet is empty

In [ ]:
best

Sample(sample={'U1': -1.1975143099363095e-13, 'U2': 0.23669475373605475, 'U3': 0.937599260734739, 'U4': 0.08061770729365869, 'U5': 0.10479458664724263, 'U6': 0.014269444341550655, 'x_1': 0.0, 'x_10': 1.0, 'x_100': 1.0, 'x_11': 1.0, 'x_12': 0.0, 'x_13': 0.0, 'x_14': 0.0, 'x_15': 1.0, 'x_16': 0.0, 'x_17': 0.0, 'x_18': 1.0, 'x_19': 1.0, 'x_2': 1.0, 'x_20': 1.0, 'x_21': 0.0, 'x_22': 0.0, 'x_23': 1.0, 'x_24': 0.0, 'x_25': 0.0, 'x_26': 0.0, 'x_27': 0.0, 'x_28': 1.0, 'x_29': 0.0, 'x_3': 0.0, 'x_30': 0.0, 'x_31': 1.0, 'x_32': 1.0, 'x_33': 0.0, 'x_34': 1.0, 'x_35': 0.0, 'x_36': 1.0, 'x_37': 0.0, 'x_38': 1.0, 'x_39': 0.0, 'x_4': 1.0, 'x_40': 1.0, 'x_41': 0.0, 'x_42': 0.0, 'x_43': 0.0, 'x_44': 0.0, 'x_45': 1.0, 'x_46': 1.0, 'x_47': 1.0, 'x_48': 0.0, 'x_49': 0.0, 'x_5': 1.0, 'x_50': 1.0, 'x_51': 1.0, 'x_52': 0.0, 'x_53': 0.0, 'x_54': 1.0, 'x_55': 1.0, 'x_56': 0.0, 'x_57': 0.0, 'x_58': 0.0, 'x_59': 0.0, 'x_6': 0.0, 'x_60': 0.0, 'x_61': 1.0, 'x_62': 1.0, 'x_63': 1.0, 'x_64': 1.0, 'x_65': 1.0, 'x_66'

In [ ]:
selected_group = [(key, val) for key, val in best.sample.items() if 'x' in key]
group1 = np.ones(n)
for idx, val in selected_group:
    group1[int(idx.split('_')[1])-1] = val

In [ ]:
group2 = np.ones(n) - group1

In [ ]:
ClinicalTrial(w=w).discrepancy(group1, group2)

1.4843168704933296